# Getting Started with Qrisp

*A comprehensive introduction to quantum programming with Qrisp*

---

## About This Notebook

This notebook provides a hands-on introduction to **Qrisp**, a high-level quantum programming framework developed in Europe. You'll learn:

1. How to create and manipulate QuantumVariables
2. Implementing Grover's algorithm to solve equations
3. Quantum Phase Estimation (QPE)
4. **Running algorithms on real IQM quantum hardware**

**Qrisp** abstracts away low-level qubit operations, letting you work with variables and functions instead of qubits and circuits.

---

## Environment Setup

### Prerequisites

- Python 3.10, 3.11, or 3.12 (Qrisp not yet compatible with 3.13+)
- `uv` installed (recommended) or `pip`

### Step 1: Install `uv`

If you don't have `uv` installed yet, pick the easiest option for your system:

**macOS or Linux:**
```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

**Windows:**
Open PowerShell and run:
```powershell
powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"
```

**Already have Homebrew on Mac?**
```bash
brew install uv
```

**Check it worked:**
```bash
uv --version
```

### Step 2: Install Dependencies

**Using `uv` (Recommended):**

Navigate to the project root and run:

```bash
# Go to the project root
cd /path/to/qas2025

# Install all dependencies (Qrisp, Jupyter, and everything else)
uv sync

# Start Jupyter notebook
uv run jupyter notebook

# Navigate to: notebooks/getting-started/qrisp-intro.ipynb
```

**Using `pip`:**

If you prefer traditional pip:

```bash
# Create a virtual environment
python3.12 -m venv venv
source venv/bin/activate  # On macOS/Linux
# OR
venv\Scripts\activate     # On Windows

# Install dependencies
pip install qrisp jupyter notebook ipykernel

# For IQM hardware access:
pip install qrisp[iqm]

# Start Jupyter
jupyter notebook
```

---

## IQM Resonance Hardware Access

QAS2025 participants have access to **real IQM quantum computers** via IQM Resonance!

### Your Authentication Token

Your authentication token is stored in `token.txt` (same directory as this notebook). This token:
- Provides access to IQM quantum hardware
- Is gitignored for security
- Should never be shared publicly

### What You'll Do

In this notebook, you'll:
1. **First**: Run algorithms using simulation (fast, for learning)
2. **Then**: Run the same algorithms on **real IQM quantum hardware**
3. **Compare**: Simulation vs real hardware results

Let's get started!

---

## Section 1: Creating and Working with QuantumVariables

### What is a QuantumVariable?

A `QuantumVariable` is Qrisp's fundamental abstraction. Unlike working directly with qubits, you can:
- Create variables that represent quantum states
- Apply gates using Pythonic syntax
- Automatically manage qubit resources

Let's start with the basics:

In [1]:
from qrisp import QuantumVariable

# Create a QuantumVariable with 5 qubits
qv = QuantumVariable(5)

print(f"Created QuantumVariable with {qv.size} qubits")
print(f"Variable name: {qv}")

Created QuantumVariable with 5 qubits
Variable name: {'00000': 1.0}                                                        


### Applying Quantum Gates

Qrisp provides intuitive gate operations:

In [2]:
from qrisp import h, z, cx

# Apply Hadamard to first qubit
h(qv[0])

# Apply Z-gate to all qubits
z(qv)

# Apply CNOT with control on qubit 0, target on qubit 3
cx(qv[0], qv[3])

print("Gates applied successfully!")

Gates applied successfully!


### Viewing the Quantum Circuit

Every QuantumVariable is associated with a QuantumSession (`.qs`) that tracks the circuit:

In [3]:
# Display the circuit
print(qv.qs)

QuantumCircuit:
---------------
      ┌───┐┌───┐     
qv.0: ┤ H ├┤ Z ├──■──
      ├───┤└───┘  │  
qv.1: ┤ Z ├───────┼──
      ├───┤       │  
qv.2: ┤ Z ├───────┼──
      ├───┤     ┌─┴─┐
qv.3: ┤ Z ├─────┤ X ├
      ├───┤     └───┘
qv.4: ┤ Z ├──────────
      └───┘          
Live QuantumVariables:
----------------------
QuantumVariable qv


### Working with QuantumChar

Qrisp provides specialized types like `QuantumChar` for character data:

In [4]:
from qrisp import QuantumChar

# Create and initialize a QuantumChar
qch = QuantumChar()
qch[:] = "e"  # Initialize to character 'e'

# Measure and see the result
print("Initial state:")
print(qch)

Initial state:
{'e': 1.0}                                                                           


### Creating Entanglement

Let's entangle the QuantumChar with our previous variable:

In [5]:
# Entangle qch with qv using CNOT
cx(qv[0], qch[0])

# Now the QuantumChar is in superposition
print("After entanglement:")
print(qch)

After entanglement:
{'e': 0.5, 'f': 0.5}                                                                 


### Measurement and State Inspection

In [6]:
# Get measurement results
results = qch.get_measurement()
print("Measurement results:")
print(results)

# View the statevector (for small systems)
print("\nStatevector:")
print(qch.qs.statevector())

Measurement results:                                                                 
{'e': 0.5, 'f': 0.5}

Statevector:
sqrt(2)*(|00000>*|e> - |10010>*|f>)/2                                                


### Cleanup Resources

Qrisp allows you to free qubits when they're no longer needed:

In [7]:
# Delete the variable (only works if all qubits are in |0⟩ state)
# Note: This might fail due to entanglement - that's expected!
try:
    qv.delete(verify=True)
    print("Qubits successfully freed")
except:
    print("Cannot delete - qubits are entangled or not in |0⟩ state")

Cannot delete - qubits are entangled or not in |0⟩ state                             


---

## Section 2: Solving Quadratic Equations with Grover's Algorithm

### Problem Statement

We'll use Grover's algorithm to solve: **x² = 0.25**

The solutions are x = ±0.5. Let's see how Qrisp makes this elegant!

### Step 1: Define the Oracle

The oracle marks states that satisfy our equation:

In [8]:
from qrisp import auto_uncompute, z, h, QuantumFloat

@auto_uncompute
def sqrt_oracle(qf):
    """Oracle that marks solutions to x² = 0.25"""
    # This creates a temporary quantum boolean
    temp_qbool = qf * qf == 0.25
    # Mark the solution with a phase flip
    z(temp_qbool)
    # The @auto_uncompute decorator automatically cleans up temp_qbool!

print("Oracle defined successfully!")
print("The @auto_uncompute decorator handles ancilla management automatically.")

Oracle defined successfully!
The @auto_uncompute decorator handles ancilla management automatically.


### Step 2: Implement Grover's Algorithm

In [9]:
from math import pi
from qrisp.grover import diffuser

# Create a signed QuantumFloat with 3 qubits and precision -1
# This gives us range [-4, 3.5] with step 0.5
qf = QuantumFloat(3, -1, signed=True)

# Calculate optimal number of Grover iterations
n = qf.size
num_solutions = 2  # We expect ±0.5
iterations = int(0.25 * pi * (2**n / num_solutions) ** 0.5)

print(f"QuantumFloat created with {n} qubits")
print(f"Running {iterations} Grover iterations")

# Create uniform superposition
h(qf)

# Grover iteration: Oracle + Diffuser
for i in range(iterations):
    sqrt_oracle(qf)  # Mark solutions
    diffuser(qf)     # Amplify marked states

print("\nGrover's algorithm complete!")

QuantumFloat created with 4 qubits
Running 2 Grover iterations

Grover's algorithm complete!


### Step 3: Measure and Analyze Results

In [10]:
# Measure the QuantumFloat
results = qf.get_measurement()

print("Results from Grover's algorithm:")
print("================================")
for value, probability in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"x = {value:5.1f}  |  Probability: {probability:.1%}")

print("\nExpected: ~47% probability for both x = 0.5 and x = -0.5")

Results from Grover's algorithm:                                                     
x =   0.5  |  Probability: 47.3%
x =  -0.5  |  Probability: 47.3%
x =   0.0  |  Probability: 0.4%
x =   1.0  |  Probability: 0.4%
x =   1.5  |  Probability: 0.4%
x =   2.0  |  Probability: 0.4%
x =   2.5  |  Probability: 0.4%
x =   3.0  |  Probability: 0.4%
x =   3.5  |  Probability: 0.4%
x =  -4.0  |  Probability: 0.4%
x =  -3.5  |  Probability: 0.4%
x =  -3.0  |  Probability: 0.4%
x =  -2.5  |  Probability: 0.4%
x =  -2.0  |  Probability: 0.4%
x =  -1.5  |  Probability: 0.4%
x =  -1.0  |  Probability: 0.4%

Expected: ~47% probability for both x = 0.5 and x = -0.5


### Visualize the Circuit

In [11]:
# View the compiled circuit
print("Circuit depth:", qf.qs.depth())
print("Number of qubits:", qf.qs.num_qubits())
print("\nCircuit structure:")
print(qf.qs)

Circuit depth: 2664
Number of qubits: 38

Circuit structure:
QuantumCircuit:
---------------
                  ┌───┐┌───────────┐                            »
            qf.0: ┤ H ├┤0          ├────────────────────────────»
                  ├───┤│           │                            »
            qf.1: ┤ H ├┤1          ├────────────────────────────»
                  ├───┤│           │                            »
            qf.2: ┤ H ├┤2          ├────────────────────────────»
                  ├───┤│           │                            »
            qf.3: ┤ H ├┤3          ├────────────────────────────»
                  └───┘│           │┌────────┐     ┌───────────┐»
       mul_res.0: ─────┤4          ├┤0       ├─────┤0          ├»
                       │           ││        │     │           │»
       mul_res.1: ─────┤5          ├┤1       ├─────┤1          ├»
                       │           ││        │     │           │»
       mul_res.2: ─────┤6          ├┤2       ├───

---

## Section 3: Quantum Phase Estimation (QPE)

### What is QPE?

Quantum Phase Estimation is a fundamental quantum algorithm that estimates the phase φ when:
- Given a unitary operator U
- Given an eigenstate |ψ⟩ where U|ψ⟩ = e^(i2πφ)|ψ⟩
- QPE finds the value of φ

### Implementing QPE in Qrisp

In [12]:
from qrisp import QuantumFloat, control, QFT, h

def QPE(psi, U, precision):
    """
    Quantum Phase Estimation
    
    Parameters:
    -----------
    psi : QuantumVariable
        The eigenstate
    U : callable
        The unitary operator (as a Python function)
    precision : int
        Number of bits for phase precision
    
    Returns:
    --------
    QuantumFloat
        The estimated phase
    """
    # Create a QuantumFloat to store the phase
    res = QuantumFloat(precision, -precision)
    
    # Create superposition in the result register
    h(res)
    
    # Controlled-U operations
    for i in range(precision):
        with control(res[i]):
            # Apply U 2^i times
            for j in range(2**i):
                U(psi)
    
    # Inverse QFT to extract the phase
    return QFT(res, inv=True)

print("QPE function defined!")
print("Notice how U is a Python function, not a circuit object - that's Qrisp magic!")

QPE function defined!
Notice how U is a Python function, not a circuit object - that's Qrisp magic!


### Testing QPE with a Simple Example

In [13]:
from qrisp import p, QuantumVariable, multi_measurement
import numpy as np

# Define a simple unitary with known phases
def U(psi):
    """Apply phase rotations to each qubit"""
    phi_1 = 0.5    # Phase for first qubit
    phi_2 = 0.125  # Phase for second qubit
    
    p(phi_1 * 2 * np.pi, psi[0])  # Phase gate
    p(phi_2 * 2 * np.pi, psi[1])  # Phase gate

# Create the eigenstate
psi = QuantumVariable(2)
h(psi)  # Create superposition

# Run QPE with 3-bit precision
res = QPE(psi, U, precision=3)

print("Running QPE...")

Running QPE...


### Analyze QPE Results

In [14]:
# Measure both the eigenstate and estimated phase
results = multi_measurement([psi, res])

print("QPE Results:")
print("="*50)
print(f"{'Eigenstate':<15} | {'Estimated Phase':<15} | Probability")
print("="*50)

for (psi_val, phase_val), prob in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{str(psi_val):<15} | {phase_val:>14.3f} | {prob:>10.1%}")

print("\nExpected phases: 0.5 and 0.125")
print("The algorithm should find these values with high probability!")

QPE Results:                                                                         
Eigenstate      | Estimated Phase | Probability
00              |          0.000 |      25.0%
10              |          0.500 |      25.0%
01              |          0.125 |      25.0%
11              |          0.625 |      25.0%

Expected phases: 0.5 and 0.125
The algorithm should find these values with high probability!


---

## Running on Real Quantum Hardware

Now that we've learned the basics with simulation, let's run our algorithms on **real IQM quantum computers**!

### Understanding Simulation vs Real Hardware

**Simulation (what we've been doing):**
- Runs on your classical computer
- Fast and deterministic
- Perfect for learning and debugging
- No noise or errors

**Real Quantum Hardware:**
- Runs on actual quantum processors
- Subject to quantum noise and decoherence
- Provides real-world quantum computing experience
- Results show effects of real quantum phenomena

Let's compare both approaches!

---

### Step 1: Load Your IQM Token

First, let's load your authentication token from the file:

In [15]:
# Load your IQM authentication token
with open('token.txt', 'r') as f:
    iqm_token = f.read().strip()

print("✓ Token loaded successfully!")
print(f"✓ Token length: {len(iqm_token)} characters")

✓ Token loaded successfully!
✓ Token length: 64 characters


### Step 2: Configure IQM Backend

Qrisp provides a convenient interface to IQM quantum computers. Let's set it up:

In [20]:
# Import IQM backend support
# Note: This requires qrisp[iqm] to be installed
try:
    from qrisp.interface import IQMBackend
    
    # Configure the IQM backend
    # Available IQM systems through Resonance:
    # - garnet (20-qubit system)
    # Check https://resonance.meetiqm.com for available devices
    
    iqm_backend = IQMBackend(
        api_token=iqm_token,
        device_instance="garnet"  # IQM 20-qubit system
    )
    
    print("✓ IQM Backend configured successfully!")
    print(f"✓ Connected to: garnet (IQM 20-qubit system)")
    print("✓ Ready to run quantum circuits on real hardware!")
    
except ImportError:
    print("⚠ IQM backend not available")
    print("To install: pip install qrisp[iqm]")
    iqm_backend = None
except Exception as e:
    print(f"⚠ Could not connect to IQM: {e}")
    print("Check your token and internet connection")
    iqm_backend = None

⚠ IQM backend not available
To install: pip install qrisp[iqm]


In [18]:
!pip install 'qrisp[iqm]'

/Users/karim/.pyenv/versions/3.12.7/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of iqm-exa-common to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of iqm-exa-common to determine which version is compatible with other requirements. This could take a while.
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of iqm-pulse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of iqm-pulse to determine which version is compatible w

### Step 3: Run a Simple Circuit - Simulation vs Real Hardware

Let's create a simple quantum circuit and run it both ways to see the difference:

In [19]:
from qrisp import QuantumVariable, h

# Create a simple 2-qubit circuit
qv_test = QuantumVariable(2)
h(qv_test)  # Put both qubits in superposition

print("Circuit created:")
print(qv_test.qs)
print("\n" + "=" * 60)

# Run on SIMULATION (default)
print("\n📊 SIMULATION Results:")
print("-" * 60)
results_sim = qv_test.get_measurement()
for state, prob in sorted(results_sim.items()):
    print(f"  |{state}⟩  →  {prob:.1%}")

# Run on REAL IQM HARDWARE
if iqm_backend is not None:
    print("\n🔬 REAL IQM HARDWARE Results:")
    print("-" * 60)
    print("⏳ Submitting job to IQM Garnet...")
    
    results_hardware = qv_test.get_measurement(backend=iqm_backend, shots=1000)
    
    for state, prob in sorted(results_hardware.items()):
        print(f"  |{state}⟩  →  {prob:.1%}")
    
    print("\n💡 Notice the differences! Real hardware shows:")
    print("   - Measurement noise")
    print("   - Quantum decoherence effects")
    print("   - Slight deviations from ideal 25% for each state")
else:
    print("\n⚠ Skipping hardware run (IQM backend not configured)")
    print("Make sure to install: pip install qrisp[iqm]")

Circuit created:
QuantumCircuit:
---------------
           ┌───┐
qv_test.0: ┤ H ├
           ├───┤
qv_test.1: ┤ H ├
           └───┘
Live QuantumVariables:
----------------------
QuantumVariable qv_test


📊 SIMULATION Results:
------------------------------------------------------------
  |00⟩  →  25.0%                                                                     
  |01⟩  →  25.0%
  |10⟩  →  25.0%
  |11⟩  →  25.0%

⚠ Skipping hardware run (IQM backend not configured)
Make sure to install: pip install qrisp[iqm]


## Summary and Next Steps

### What We Covered

✅ **QuantumVariables**: High-level quantum state management

✅ **Grover's Algorithm**: Solving quadratic equations with quantum search

✅ **Quantum Phase Estimation**: Estimating eigenvalues of unitary operators

✅ **Simulation**: Running algorithms on classical simulators

✅ **Real Quantum Hardware**: Executing on IQM quantum computers via Resonance

### Key Takeaways

1. **High-level abstractions**: Qrisp lets you think in terms of variables, not qubits
2. **Automatic resource management**: `@auto_uncompute` handles ancilla qubits
3. **Pythonic syntax**: Control flow, functions, and operators work naturally
4. **Simulation first**: Test and debug with fast, noiseless simulation
5. **Real hardware access**: Run the same code on actual quantum processors
6. **NISQ reality**: Real quantum computers have noise - this is expected!

### Real Quantum Computing Experience

You've now:
- 🎯 Written quantum algorithms in Qrisp
- 🔬 Run them on **real IQM quantum hardware**
- 📊 Compared simulation vs reality
- 💡 Experienced quantum noise and decoherence firsthand

This is real quantum computing in action!

### Further Learning

📚 **Official Documentation**: [https://qrisp.eu](https://qrisp.eu)

📚 **Tutorials**: [https://qrisp.eu/general/tutorial/](https://qrisp.eu/general/tutorial/)

📚 **IQM Academy**: [https://www.iqmacademy.com](https://www.iqmacademy.com)

📚 **IQM Resonance**: [https://resonance.meetiqm.com](https://resonance.meetiqm.com)

🎓 **QAS2025 Resources**: [https://enccs.github.io/qas2025/](https://enccs.github.io/qas2025/)

### Advanced Topics to Explore

**Quantum Algorithms:**
- QAOA (Quantum Approximate Optimization Algorithm)
- VQE (Variational Quantum Eigensolver) for molecular simulation
- Quantum backtracking for constraint satisfaction
- Quantum machine learning with Qrisp

**Hardware Topics:**
- Error mitigation techniques
- Circuit optimization for NISQ devices
- Benchmarking quantum algorithms
- Hybrid classical-quantum workflows

**IQM Specific:**
- Explore different IQM architectures (Garnet, Garnet)
- Learn about IQM's native gate set
- Understand superconducting qubit technology
- Job management and monitoring on Resonance

---

## Questions?

For help with Qrisp or this tutorial:
- Check the [Qrisp documentation](https://qrisp.eu)
- Visit [IQM Resonance dashboard](https://resonance.meetiqm.com)
- Join the QAS2025 discussions
- Reach out to the organizing team

**Happy Quantum Programming on Real Hardware! 🚀🔬**

In [ ]:
# We already defined the oracle and ran Grover in simulation earlier
# Now let's run the SAME algorithm on real IQM hardware

if iqm_backend is not None:
    print("Running Grover's Algorithm on IQM Garnet Quantum Computer")
    print("=" * 60)
    
    # Recreate the Grover setup
    from qrisp import QuantumFloat, h, auto_uncompute, z
    from qrisp.grover import diffuser
    from math import pi
    
    @auto_uncompute
    def sqrt_oracle(qf):
        temp_qbool = qf * qf == 0.25
        z(temp_qbool)
    
    # Create QuantumFloat
    qf_hardware = QuantumFloat(3, -1, signed=True)
    n = qf_hardware.size
    iterations = int(0.25 * pi * (2**n / 2) ** 0.5)
    
    # Apply Grover's algorithm
    h(qf_hardware)
    for i in range(iterations):
        sqrt_oracle(qf_hardware)
        diffuser(qf_hardware)
    
    print(f"✓ Circuit prepared with {iterations} Grover iterations")
    print(f"✓ Circuit has {qf_hardware.qs.depth()} gates")
    print("\n⏳ Submitting to IQM Garnet... (this may take a minute)")
    
    # Run on real hardware
    results_hw = qf_hardware.get_measurement(backend=iqm_backend, shots=1000)
    
    print("\n🔬 Results from IQM Quantum Hardware:")
    print("-" * 60)
    for value, probability in sorted(results_hw.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  x = {value:5.1f}  |  {probability:.1%}")
    
    print("\n📊 Compare with simulation results from earlier:")
    print("   Simulation: x = ±0.5 at ~47% each")
    print("   Hardware: Shows real quantum noise and error effects!")
    
else:
    print("⚠ IQM backend not available - skipping hardware run")
    print("To run on real hardware: pip install qrisp[iqm]")

### Step 4: Run Grover's Algorithm on Real Hardware

Now let's run our Grover's algorithm (solving x² = 0.25) on real IQM quantum hardware!

In [ ]:
from qrisp import QuantumVariable, h

# Create a simple 2-qubit circuit
qv_test = QuantumVariable(2)
h(qv_test)  # Put both qubits in superposition

print("Circuit created:")
print(qv_test.qs)
print("\n" + "=" * 60)

# Run on SIMULATION (default)
print("\n📊 SIMULATION Results:")
print("-" * 60)
results_sim = qv_test.get_measurement()
for state, prob in sorted(results_sim.items()):
    print(f"  |{state}⟩  →  {prob:.1%}")

# Run on REAL IQM HARDWARE
if iqm_backend is not None:
    print("\n🔬 REAL IQM HARDWARE Results:")
    print("-" * 60)
    print("⏳ Submitting job to IQM Garnet...")
    
    results_hardware = qv_test.get_measurement(backend=iqm_backend, shots=1000)
    
    for state, prob in sorted(results_hardware.items()):
        print(f"  |{state}⟩  →  {prob:.1%}")
    
    print("\n💡 Notice the differences! Real hardware shows:")
    print("   - Measurement noise")
    print("   - Quantum decoherence effects")
    print("   - Slight deviations from ideal 25% for each state")
else:
    print("\n⚠ Skipping hardware run (IQM backend not configured)")
    print("Make sure to install: pip install qrisp[iqm]")

### IQM Hardware Access (For QAS2025 Participants)

To run on IQM quantum computers, you need:
1. Install IQM support: `pip install qrisp[iqm]` or add to `pyproject.toml`
2. Load your authentication token
3. Configure the IQM backend

**Note**: The code below is for reference - you'll need proper IQM credentials.

In [ ]:
# Example IQM integration (requires qrisp[iqm] and valid credentials)
"""
# Load token from file
with open('token.txt', 'r') as f:
    token = f.read().strip()

# Configure IQM backend
from qrisp.interface import IQMBackend

iqm_backend = IQMBackend(
    server_url="https://cocos.resonance.meetiqm.com/garnet",
    token=token
)

# Run your quantum circuit
results = qf.get_measurement(backend=iqm_backend)
"""

print("IQM integration code provided above (commented out)")
print("Uncomment and configure with your credentials to use real hardware!")

---

## Summary and Next Steps

### What We Covered

✅ **QuantumVariables**: High-level quantum state management

✅ **Grover's Algorithm**: Solving quadratic equations with quantum search

✅ **Quantum Phase Estimation**: Estimating eigenvalues of unitary operators

✅ **Hardware Integration**: Exporting to Qiskit and IQM backends

### Key Takeaways

1. **High-level abstractions**: Qrisp lets you think in terms of variables, not qubits
2. **Automatic resource management**: `@auto_uncompute` handles ancilla qubits
3. **Pythonic syntax**: Control flow, functions, and operators work naturally
4. **Hardware ready**: Easy integration with real quantum computers

### Further Learning

📚 **Official Documentation**: [https://qrisp.eu](https://qrisp.eu)

📚 **Tutorials**: [https://qrisp.eu/general/tutorial/](https://qrisp.eu/general/tutorial/)

📚 **IQM Academy**: [https://www.iqmacademy.com](https://www.iqmacademy.com)

🎓 **QAS2025 Resources**: Check the main documentation site

### Advanced Topics to Explore

- QAOA (Quantum Approximate Optimization Algorithm)
- Quantum backtracking for constraint satisfaction
- VQE (Variational Quantum Eigensolver) for molecular simulation
- Quantum machine learning with Qrisp

---

## Questions?

For help with Qrisp or this tutorial:
- Check the [Qrisp documentation](https://qrisp.eu)
- Join the QAS2025 discussions
- Reach out to the organizing team

**Happy Quantum Programming! 🚀**